In [ ]:
!pip install torch transformers flask beautifulsoup4 requests nltk

In [ ]:
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install torchviz


  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=56bc3e270a7fcb1151fa553abb1fbe6646adce363f3bfeaa3a43f3fdb6321834
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [ ]:
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
DIR = '/content/drive/MyDrive/Analytics/'

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import json

# FAQ dataset
path = '/content/drive/MyDrive/Analytics/University_data.json'
with open(path, 'r') as file:
    faq_data = json.load(file)

# Combine all FAQ pairs into a single dataset
dataset_list = []
label_mapping = {}  # Mapping from string labels to numerical values
label_index = 0

for category, faqs in faq_data.items():
    for faq in faqs:
        question = faq["question"]
        answer = faq["answer"]
        label = category  # Use the category as the label for this example
        if label not in label_mapping:
            label_mapping[label] = label_index
            label_index += 1

        dataset_list.append({"text": f"{category}: {question}", "label": label_mapping[label]})

# Create a Hugging Face datasets.Dataset directly from the list
dataset = Dataset.from_dict({"text": [item["text"] for item in dataset_list], "label": [item["label"] for item in dataset_list]})

# Split the dataset into train and validation (adjust split ratio as needed)
split_ratio = 0.8
train_dataset = dataset.select(range(int(split_ratio * len(dataset))))
validation_dataset = dataset.select(range(int(split_ratio * len(dataset)), len(dataset)))

# BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenize input texts within the datasets with explicit padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt", max_length=128)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_function, batched=True)

# Set up training arguments and trainer
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_steps=100,
)

# Ensure labels are properly set
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

tokenized_validation_dataset = tokenized_validation_dataset.remove_columns(["text"])
tokenized_validation_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# BERT model and tokenizer
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(label_mapping))

# Set up training arguments and trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Map:   0%|          | 0/516 [00:00<?, ? examples/s]

Map:   0%|          | 0/130 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss
500,0.013700,4.983867


TrainOutput(global_step=660, training_loss=0.2832737333846815, metrics={'train_runtime': 73.9241, 'train_samples_per_second': 139.603, 'train_steps_per_second': 8.928, 'total_flos': 679009370112000.0, 'train_loss': 0.2832737333846815, 'epoch': 20.0})


{'eval_loss': 5.036649227142334, 'eval_runtime': 0.2894, 'eval_samples_per_second': 449.198, 'eval_steps_per_second': 31.098, 'epoch': 20.0}


In [ ]:

model_name = "Epoch_20"
model_path = f"/content/drive/MyDrive/Analytics/BERTModels/{model_name}"

# Save the trained model with the specified name
model.save_pretrained(model_path)


In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from torchviz import make_dot

# Load pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Example input text
input_text = "Hello, how are you today?"

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt")

# Perform forward pass to get model output
outputs = model(**inputs)

# Visualize the model architecture
dot = make_dot(outputs.logits, params=dict(model.named_parameters()))

# Save the visualization to a file (e.g., PNG)
dot.render(DIR+"model_visualization", format="png", cleanup=True)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'/content/drive/MyDrive/Analytics/model_visualization.png'

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import json

# model_path= '/content/drive/MyDrive/Analytics/model1Epoch'
# FAQ dataset
faq_path = '/content/drive/MyDrive/Analytics/University_data.json'

def load_model_and_tokenizer(model_path):
    model = BertForSequenceClassification.from_pretrained(model_path)
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    return model, tokenizer

def load_faq_data(faq_path):
    with open(faq_path, 'r') as file:
        faq_data = json.load(file)
    return faq_data

def get_inverse_label_mapping(label_mapping):
    return {v: k for k, v in label_mapping.items()}

def get_faq_answer(predicted_category, faq_data):
    return faq_data.get(predicted_category, [{"answer": "I'm sorry, I don't have an answer for that."}])[0]["answer"]

def chatbot_response(user_input, model, tokenizer, inverse_label_mapping, faq_data):
    # Tokenize user input
    tokenized_input = tokenizer(user_input, padding="max_length", truncation=True, return_tensors="pt", max_length=128)

    # Model inference
    with torch.no_grad():
        model_output = model(**tokenized_input)

    # Get predicted class
    predicted_class = torch.argmax(model_output.logits, dim=1).item()

    # Map predicted class to FAQ category
    predicted_category = inverse_label_mapping.get(predicted_class, "unknown_category")

    # Retrieve FAQ answer
    faq_answer = get_faq_answer(predicted_category, faq_data)

    return faq_answer

if __name__ == "__main__":

    label_mapping = {category: idx for idx, category in enumerate(faq_data.keys())}

    model, tokenizer = load_model_and_tokenizer(model_path)
    faq_data = load_faq_data(faq_path)
    inverse_label_mapping = get_inverse_label_mapping(label_mapping)

    print("Chatbot: Hello! Ask me anything, type 'exit' to end.")

    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        response = chatbot_response(user_input, model, tokenizer, inverse_label_mapping, faq_data)
        print("Chatbot:", response)


Chatbot: Hello! Ask me anything, type 'exit' to end.
User: How to purchase parking permits
Chatbot:  Permits can be purchased through the Permits can be purchased through the Parking Portal Website Parking Portal Website. 
User: How to charge my electric vehicle
Chatbot: Athletic trainers are healthcare professionals who provide prevention services, emergency care, examination and clinical diagnosis, therapeutic intervention and rehabilitation of injuries and illnesses
User: What is the cost to  charge my electric vehicle
Chatbot:  Permits can be purchased through the Permits can be purchased through the Parking Portal Website Parking Portal Website. 
User: I lost my lost pacificcard
Chatbot:  If your PacificCard has been lost or stolen, please login and select the Lost/Stolen cards link to deactivate your accounts. Allow others to see your PacificCard
User: Where are you located?
Chatbot:  Permits can be purchased through the Permits can be purchased through the Parking Portal Website